<div align="center">
  <span style="color: white; font-size: 45px;">
    Image Classification
  </span>
</div>

## Import Library

In [52]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Define transformations to apply to each image

In [53]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),      # Resize images to 64x64 pixels
    transforms.ToTensor(),            # Convert to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize pixel values to range [-1, 1]
])

## Import dataset

In [67]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
import json
import numpy as np
import os

# Load the nutritional data from the JSON file
with open('D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/nutritional_data.json', 'r') as f:
    nutritional_data = json.load(f)

# Define image size
IMG_SIZE = (150, 150)

# Paths to dataset
train_dir = 'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train'
test_dir = 'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test'

# Create ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Generate data for training and testing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical'
)

# Print class labels detected by the generator
print("Class labels in the training data:")
print(train_generator.class_indices)

# Define the model architecture with a second output for the quantity
def create_model():
    input_layer = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    
    # Convolutional layers
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Flatten the output from the convolutional layers
    x = Flatten()(x)
    
    # Class prediction output
    class_output = Dense(len(train_generator.class_indices), activation='softmax', name='food_class')(x)
    
    # Quantity prediction output (assuming regression for quantity prediction)
    quantity_output = Dense(1, activation='linear', name='food_quantity')(x)
    
    # Create the final model with multiple outputs
    model = Model(inputs=input_layer, outputs=[class_output, quantity_output])
    
    return model

# Compile the model with two loss functions
model = create_model()
model.compile(optimizer=Adam(), 
              loss={'food_class': 'categorical_crossentropy', 'food_quantity': 'mse'},
              metrics={'food_class': 'accuracy', 'food_quantity': 'mae'})

# Train the model (You can add validation data and epochs as needed)
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Save the trained model
model.save('food_recognition_with_quantity_model.h5')


Found 465 images belonging to 3 classes.


Found 193 images belonging to 3 classes.
Class labels in the training data:
{'ahmok': 0, 'apple': 1, 'orange': 2}
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 284ms/step - food_class_accuracy: 0.5257 - loss: 0.8228 - val_food_class_accuracy: 0.8135 - val_loss: 0.5990
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 275ms/step - food_class_accuracy: 0.8497 - loss: 0.4533 - val_food_class_accuracy: 0.8290 - val_loss: 0.3950
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 280ms/step - food_class_accuracy: 0.9304 - loss: 0.2005 - val_food_class_accuracy: 0.9326 - val_loss: 0.3292
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 281ms/step - food_class_accuracy: 0.9478 - loss: 0.1662 - val_food_class_accuracy: 0.9067 - val_loss: 0.3450
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 282ms/step - food_class_accuracy: 0.9520 - loss: 0.1459 - val_food_class_accuracy: 0.9119 - val_loss: 0.3106
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 292ms/step - food_class_accuracy: 0.9575 - loss: 0.1258 - val_food_class_accuracy: 0.9171 - val

In [69]:
def predict_image(img_url, nutrition_data):
    img_array = load_image_from_url(img_url, target_size=(150, 150))
    
    # Load the trained model
    model = tf.keras.models.load_model('food_recognition_with_quantity_model.h5')
    
    # Get predictions (food class and quantity)
    food_class_pred, food_quantity_pred = model.predict(img_array)
    
    food_class = np.argmax(food_class_pred, axis=1)
    food_name = list(nutrition_data.keys())[food_class[0]]  # Get the food name based on the class
    
    # Adjust nutrition based on predicted quantity
    quantity = food_quantity_pred[0][0]  # Predicted quantity
    
    food_nutrition = nutrition_data.get(food_name, None)
    
    if food_nutrition:
        adjusted_calories = food_nutrition['calories'] * quantity
        adjusted_fat = food_nutrition['fat'] * quantity
        adjusted_protein = food_nutrition['protein'] * quantity
        adjusted_sugar = food_nutrition['sugar'] * quantity
        
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Quantity: {quantity}")
        print(f"Calories: {adjusted_calories} kcal")
        print(f"Fat: {adjusted_fat} g")
        print(f"Protein: {adjusted_protein} g")
        print(f"Sugar: {adjusted_sugar} g")
    else:
        print("Nutrition data not available for this food.")


## Multiple taks

In [55]:
def build_multi_task_model():
    # Input layer for the images
    image_input = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    # Convolutional layers for feature extraction
    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)

    # Flatten the layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output for food classification (categorical classification)
    food_output = Dense(len(nutritional_data), activation='softmax', name='food_classification')(x)

    # Output for nutritional predictions (regression for 4 values: calories, fat, protein, sugar)
    nutrition_output = Dense(4, activation='linear', name='nutrition_prediction')(x)

    # Define the model with two outputs
    model = Model(inputs=image_input, outputs=[food_output, nutrition_output])

    return model


## Model Computing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Use target_size=(224, 224) to resize images to the required size
train_generator = train_datagen.flow_from_directory(
    'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train',  # Path to your training data directory
    target_size=(224, 224),  
    batch_size=32,
    class_mode='categorical'  
)

test_generator = test_datagen.flow_from_directory(
    'D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test',  # Path to your testing data directory
    target_size=(224, 224),  # Resize images to (224, 224)
    batch_size=32,
    class_mode='categorical'  # Change as needed depending on your task
)


Found 614 images belonging to 12 classes.
Found 194 images belonging to 4 classes.


In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load Pretrained Model (MobileNetV2)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (keep pretrained weights)
base_model.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
num_classes = len(train_generator.class_indices)  # Get number of classes
output_layer = Dense(num_classes, activation="softmax")(x)


# Create the model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory("D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train", target_size=(224, 224), batch_size=32, class_mode="categorical")
test_generator = test_datagen.flow_from_directory("D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test", target_size=(224, 224), batch_size=32, class_mode="categorical")

# Train the model
epochs = 15

history = model.fit(train_generator,  
    validation_data=test_generator,  
    epochs=epochs
)

# Save the model
model.save('food_recognition_nutrition_model.h5')

print("Model saved successfully.")


Found 563 images belonging to 12 classes.
Found 606 images belonging to 12 classes.
Epoch 1/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.2414 - loss: 2.6958 - val_accuracy: 0.4472 - val_loss: 1.5818
Epoch 2/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.4835 - loss: 1.6220 - val_accuracy: 0.6997 - val_loss: 1.1808
Epoch 3/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.5973 - loss: 1.3188 - val_accuracy: 0.7277 - val_loss: 0.9818
Epoch 4/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7116 - loss: 1.0357 - val_accuracy: 0.7442 - val_loss: 0.8402
Epoch 5/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7514 - loss: 0.8583 - val_accuracy: 0.7624 - val_loss: 0.7427
Epoch 6/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7560 - loss: 0.7781 - val_accuracy: 0.7937 - val_loss: 0.6805
Epoch 7/15
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8149 - loss: 0.6278 - val_accuracy: 0.8119 - val_loss: 0.6305
Epoch 8/15
18/18 ━━━━━━━━━━━━━━━━━

Model saved successfully.


## Model Training

In [16]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()  # Enable eager execution if not already enabled


In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import json

# Load the trained model
model = tf.keras.models.load_model("food_recognition_nutrition_model.h5")

# Load class labels (from train_generator)
class_labels = {
    0: 'Amok',
    1: 'Apple',
    2: 'BorBor',
    3: 'Cha Loc Lac',
    4: 'Chicken Rice',
    5: 'Lemongrass BBQ Beef Skewers',
    6: 'Lort Cha',
    7: 'Nom Banh Chok',
    8: 'Omelette Rice with Pork',
    9: 'Orange Fruit',
    10: 'Red Curry Coconut Wings',
    11: 'Samlar KorKour'
}

# Load nutrition data from JSON file
def load_nutrition_info(json_file="nutritional_data.json"):
    with open(json_file, "r") as file:
        return json.load(file)

nutrition_data = load_nutrition_info()


In [3]:
# Function to preprocess and predict the image
def predict_food(image_path):
    # Load image and preprocess
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the highest probability class index
    food_name = class_labels[predicted_class]

    # Get nutrition info
    nutrition_info = nutrition_data.get(food_name, {"calories": "N/A", "fat": "N/A", "protein": "N/A", "sugar": "N/A"})

    # Print results
    print(f"Predicted Food: {food_name}")
    print(f"Calories: {nutrition_info['calories']} kcal")
    print(f"Fat: {nutrition_info['fat']} g")
    print(f"Protein: {nutrition_info['protein']} g")
    print(f"Sugar: {nutrition_info['sugar']} g")

    return food_name, nutrition_info


In [12]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('food_recognition_nutrition_model.h5')

# Load class labels (same order as training)
class_labels = [
    "Amok", "apple", "BorBor", "Cha Loc Lac", "Chikhen Rice", "Lemongrass BBQ Beef Skewers", 
    "Lort Cha", "Nom Banh Chok", "Omelete Rice with Pork", "Orage Fruit", 
    "Red Curry Coconut Wings", "Samlar KorKour"
]

# Load Nutrition Data
def load_nutrition_info(json_file="nutritional_data.json"):
    with open(json_file, "r") as file:
        data = json.load(file)

    # Convert list to dictionary for easy lookup
    return {item["name"]: {"calories": item["calories"], "carbs": item["carbs"], "fat": item["fat"], "protein": item["protein"]} for item in data}

nutrition_data = load_nutrition_info()

# Function to preprocess and predict food from an image
def predict_food(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  # Get index of highest probability class
    food_name = class_labels[predicted_class]

    # Get nutrition info
    nutrition_info = nutrition_data.get(food_name, {"calories": "N/A", "carbs": "N/A", "fat": "N/A", "protein": "N/A"})

    # Print results
    print(f"Predicted Food: {food_name}")
    print(f"Calories: {nutrition_info['calories']} kcal")
    print(f"Carbs: {nutrition_info['carbs']} g")
    print(f"Fat: {nutrition_info['fat']} g")
    print(f"Protein: {nutrition_info['protein']} g")

# Example Test
image_path = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/test/Nom Banh Chok/6168441767_2032207878_b.jpg"  # Change this to your test image path

predict_food(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 842ms/step
Predicted Food: Lemongrass BBQ Beef Skewers
Calories: 611.65 kcal
Carbs: 13.43 g
Fat: 3.5 g
Protein: 88.49 g


In [ ]:
# Provide the path of an image to test

# Predict
predict_food(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


AttributeError: 'list' object has no attribute 'get'

In [36]:
def predict_image(img_url, nutrition_data, class_labels):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Load the model (ensure you have the trained model saved as .h5)
    model = tf.keras.models.load_model('food_recognition_nutrition_model.h5')
    
    # Get the food classification prediction (if model has one output)
    food_classification_pred = model.predict(img_array)
    
    # Process the results
    food_class = np.argmax(food_classification_pred, axis=1)  # Get the class with the highest probability
    food_name = list(class_labels.keys())[food_class[0]]  # Get the food name based on the class index
    
    # Get the predicted nutrition value (adjust this according to your model structure)
    food_nutrition = nutrition_data.get(food_name, None)
    
    if food_nutrition:
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Calories: {food_nutrition['calories']} kcal")
        print(f"Fat: {food_nutrition['fat']} g")
        print(f"Protein: {food_nutrition['protein']} g")
        print(f"Sugar: {food_nutrition['sugar']} g")
    else:
        print("Nutrition data not available for this food.")


## Model Evaluation

In [34]:
def predict_image(img_url, nutrition_data, class_labels):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Load the model (ensure you have the trained model saved as .h5)
    model = tf.keras.models.load_model('food_recognition_nutrition_model.h5')
    
    # Get the food classification prediction (if model has one output)
    food_classification_pred = model.predict(img_array)
    
    # Process the results
    food_class = np.argmax(food_classification_pred, axis=1)  # Get the class with the highest probability
    food_name = list(class_labels.keys())[food_class[0]]  # Get the food name based on the class index
    
    # Get the predicted nutrition value (adjust this according to your model structure)
    food_nutrition = nutrition_data.get(food_name, None)
    
    if food_nutrition:
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Calories: {food_nutrition['calories']} kcal")
        print(f"Fat: {food_nutrition['fat']} g")
        print(f"Protein: {food_nutrition['protein']} g")
        print(f"Sugar: {food_nutrition['sugar']} g")
    else:
        print("Nutrition data not available for this food.")


In [1]:
import requests
import json
from PIL import Image
import numpy as np
import tensorflow as tf

def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def load_nutrition_info(json_file='D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

def predict_image(img_url, nutrition_data, class_labels):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Load the model (ensure you have the trained model saved as .h5)
    model = tf.keras.models.load_model('food_recognition_with_nutrition_models.h5')
    
    # Get the predictions for both outputs (food classification and nutrition prediction)
    food_classification_pred, nutrition_prediction_pred = model.predict(img_array)
    
    # Process the food classification results
    food_class = np.argmax(food_classification_pred, axis=1)  # Get the class with the highest probability
    food_name = list(class_labels.keys())[food_class[0]]  # Get the food name based on the class index
    
    # Get the predicted nutrition value
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Number of food items (For now, assuming one fruit per image; you can integrate object detection for actual counting)
    number_of_food = 3  # Change this if you want to detect multiple items, e.g., with an object detection model
    
    # Calculate total nutrition for the detected number of food items
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://5.imimg.com/data5/AK/RA/MY-68428614/apple.jpg'

# Load the nutrition data from the JSON file
nutrition_data = load_nutrition_info()

# Define the class labels for mapping prediction to food names
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this with your actual labels

# Make predictions for the image
predict_image(img_url, nutrition_data, class_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


AttributeError: 'list' object has no attribute 'get'

# Parth2

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

def build_model(num_classes=3):  # Adjust based on your actual number of classes
    # Load a pre-trained ResNet50 model (without the top classification layer)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers to retain pre-trained weights
    base_model.trainable = False
    
    # Add custom layers on top for both tasks
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    
    # Task 1: Classification head (food classification)
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    # Task 2: Quantity head (predict number of food items)
    quantity_output = layers.Dense(1, activation='linear', name='quantity_output')(x)

    # Create the final model
    model = models.Model(inputs=base_model.input, outputs=[class_output, quantity_output])
    
    model.compile(optimizer='adam',
                  loss={'class_output': 'categorical_crossentropy', 'quantity_output': 'mse'},
                  metrics={'class_output': 'accuracy', 'quantity_output': 'mae'})
    
    return model


In [79]:
import requests
import numpy as np
from PIL import Image

def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_image(img_url, model, class_labels, nutrition_data):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Make predictions
    class_pred, quantity_pred = model.predict(img_array)
    
    # Get the predicted class (food type)
    food_class = np.argmax(class_pred, axis=1)
    food_name = list(class_labels.keys())[food_class[0]]
    
    # Get the predicted quantity (how many items)
    number_of_food = round(quantity_pred[0][0])
    
    # Get the nutrition info for the predicted food
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Calculate total nutrition
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")


In [81]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def preprocess_and_augment_data(image_dir, batch_size=32, target_size=(224, 224)):
    # Initialize ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Flow images from the directory for training (make sure your data is organized in subdirectories by class)
    train_data = datagen.flow_from_directory(
        image_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_data


In [82]:
# Load your data for training
train_data = preprocess_and_augment_data(image_dir='D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train', batch_size=32)

# Build the model
model = build_model(num_classes=len(class_labels))

# Train the model
model.fit(train_data, epochs=10, steps_per_epoch=len(train_data) // 32)

# Save the trained model
model.save('food_recognition_with_nutrition_modell.h5')


Found 465 images belonging to 3 classes.


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 18s 0us/step
Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - class_output_accuracy: 0.2423 - loss: 1.9905
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.4418 - loss: 0.7671
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.5088 - loss: 0.7183
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.6046 - loss: 0.7595
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - class_output_accuracy: 0.5126 - loss: 0.7455
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - class_output_accuracy: 0.5334 - loss: 0.7170
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - class_output_accuracy: 0.6009 - loss: 0.7287
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 51s 3s/step - class_output_accuracy: 0.5573 - loss: 0.7096
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - class_output_accuracy: 0.5670 - loss: 0.7078
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - class_output_accuracy:

In [94]:
from tensorflow.keras.metrics import MeanSquaredError

# Load the trained model with custom metrics
def load_trained_model(model_path='food_recognition_with_nutrition_modell.h5'):
    model = tf.keras.models.load_model(model_path, custom_objects={'mse': MeanSquaredError()})
    return model


In [100]:
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError

# Load the trained model (with custom objects, if needed)
def load_trained_model(model_path='food_recognition_with_nutrition_modell.h5'):
    model = tf.keras.models.load_model(model_path, custom_objects={'MeanSquaredError': MeanSquaredError()})
    return model


In [105]:
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
import json  # Make sure to import json for loading the nutrition data

# Load the nutrition data (replace this with the actual path to your nutrition data file)
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

# Load an image from URL and preprocess it
def load_image_from_url(img_url, target_size=(224, 224)):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    # Resize the image to match the model input size
    img = img.resize(target_size)
    
    # Convert the image to a numpy array and normalize it
    img_array = np.array(img) / 255.0  # Normalize if necessary
    
    # Add batch dimension (model expects 4D input: batch_size, height, width, channels)
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

# Load the trained model (ensure you have the trained model saved)
def load_trained_model(model_path='food_recognition_with_nutrition_models.h5'):
    model = tf.keras.models.load_model(model_path)
    return model

# Predict the food type, quantity, and total nutrition from an image
def predict_food_and_quantity(img_url, model, class_labels, nutrition_data):
    # Load and preprocess the image
    img_array = load_image_from_url(img_url, target_size=(224, 224))
    
    # Make predictions (food classification and quantity prediction)
    food_class_pred, quantity_pred = model.predict(img_array)
    
    # Get the predicted food class (food type)
    food_class = np.argmax(food_class_pred, axis=1)
    food_name = list(class_labels.keys())[food_class[0]]  # Map index to food name
    
    # Get the predicted quantity (number of food items)
    # If quantity prediction is continuous, round it to the nearest integer
    number_of_food = max(1, round(quantity_pred[0][0]))  # Ensure quantity is at least 1
    
    # Get the nutrition info for the predicted food
    food_nutrition = nutrition_data.get(food_name, None)
    
    # Calculate total nutrition for the predicted quantity
    if food_nutrition:
        total_calories = food_nutrition['calories'] * number_of_food
        total_fat = food_nutrition['fat'] * number_of_food
        total_protein = food_nutrition['protein'] * number_of_food
        total_sugar = food_nutrition['sugar'] * number_of_food
        
        # Display the results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Predicted Food Class: {food_class[0]} (Index)")
        print(f"Number of Food Items: {number_of_food}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print("Nutrition data not available for this food.")

# Example image URL (replace with an actual image URL you want to test)
img_url = 'https://assets.clevelandclinic.org/transform/cd71f4bd-81d4-45d8-a450-74df78e4477a/Apples-184940975-770x533-1_jpg'

# Example class labels (ensure you have the correct mapping)
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this based on your actual classes

# Load the nutrition data (adjust path to your file)
nutrition_data = load_nutrition_info('nutritional_data.json')

# Load the trained model
model = load_trained_model('food_recognition_with_nutrition_models.h5')

# Predict food type, quantity, and total nutrition
predict_food_and_quantity(img_url, model, class_labels, nutrition_data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Food: Apple
Predicted Food Class: 1 (Index)
Number of Food Items: 1
Total Calories: 95 kcal
Total Fat: 0.3 g
Total Protein: 0.5 g
Total Sugar: 19 g


In [107]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def create_train_generator(directory, batch_size=32, target_size=(224, 224)):
    # Initialize the ImageDataGenerator with rescaling
    train_datagen = ImageDataGenerator(rescale=1./255)

    # Get class labels and quantity information from the directory structure
    def quantity_label_from_dir_name(dir_name):
        # You need to define how to extract the quantity (number of food items)
        # For example, if the folder name is "apple_5", extract the quantity '5'
        return int(dir_name.split('_')[-1])  # Assuming the quantity is at the end of the folder name

    # Create a custom generator to yield both image data and (class, quantity) labels
    def custom_train_generator():
        for batch_x, batch_y in train_datagen.flow_from_directory(
            directory,
            target_size=target_size,
            batch_size=batch_size,
            class_mode='categorical'  # Only food class
        ):
            # Get the directory names for the labels
            labels = batch_y
            quantity_labels = [quantity_label_from_dir_name(name) for name in os.listdir(directory)]
            
            # Yield images, food class labels, and quantity labels
            yield batch_x, {'food_class': labels, 'quantity': quantity_labels}

    return custom_train_generator()



In [110]:
import re

def quantity_label_from_dir_name(dir_name):
    # Use regex to search for a number at the end of the directory name
    match = re.search(r'(\d+)$', dir_name)  # This will match any digits at the end of the string
    
    if match:
        return int(match.group(1))  # Extract and return the number as an integer
    else:
        # If no number is found, return a default value or handle the case accordingly
        print(f"Warning: No quantity found in directory name: {dir_name}")
        return 1  # Default value, adjust as needed


In [109]:
import json

# Load the nutritional data from the JSON file
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as f:
        nutrition_data = json.load(f)
    return nutrition_data

# Extract nutrition for a food item
def get_food_nutrition(food_name, nutrition_data):
    # Use the food name to fetch nutritional information
    food_data = nutrition_data.get(food_name)
    if food_data:
        return food_data
    else:
        print(f"Warning: Nutrition data for {food_name} not found!")
        return None  # Return None or a default value if the food is not found

# Example usage
nutrition_data = load_nutrition_info('nutritional_data.json')
food_name = 'apple'  # Example food
nutrition_info = get_food_nutrition(food_name, nutrition_data)

if nutrition_info:
    print(f"Nutrition data for {food_name}: {nutrition_info}")


Nutrition data for apple: {'calories': 95, 'fat': 0.3, 'protein': 0.5, 'sugar': 19}


In [111]:
# Set the number of food classes (e.g., 3 classes: apple, orange, banana)
num_classes = 3

# Build the model
model = build_model(num_classes=num_classes)

# Create the training generator
train_generator = create_train_generator('D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train')

# Train the model
model.fit(train_generator, epochs=10, steps_per_epoch=100)  # Adjust the steps per epoch based on your dataset size


Found 465 images belonging to 3 classes.


ValueError: invalid literal for int() with base 10: 'ahmok'

In [ ]:
import torch
from PIL import Image
import numpy as np

# Load the YOLOv5 model (using PyTorch)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Use 'yolov5m' or 'yolov5l' for larger models

# Load image from URL or local path
img_url = 'https://grantourismotravels.com/wp-content/uploads/2017/05/Authentic-Fish-Amok-Recipe-Steamed-Fish-Curry-Cambodia-Copyright-2022-Terence-Carter-Grantourismo-T-500x375.jpg'
img = Image.open(requests.get(img_url, stream=True).raw)

# Run inference (detect objects in the image)
results = model(img)

# Results contains detected objects with labels, confidence, and bounding boxes
predicted_labels = results.names  # The class names
predicted_boxes = results.xywh[0]  # Predicted bounding boxes (x, y, width, height)
confidences = results.xywh[0][:, 4]  # Confidence scores
classes = results.xywh[0][:, 5].tolist()  # Class indices

# Count instances of each class (food items)
food_counts = {}
for cls in classes:
    food_name = predicted_labels[int(cls)]
    food_counts[food_name] = food_counts.get(food_name, 0) + 1

# Nutrition data for the food items (example data structure)
nutrition_data = {
    'apple': {'calories': 100, 'fat': 0.5, 'protein': 0.5, 'sugar': 19},
    'orange': {'calories': 80, 'fat': 0.3, 'protein': 1.5, 'sugar': 18},
    'grape': {'calories': 80, 'fat': 0.3, 'protein': 1.5, 'sugar': 18}
    
    
}

# Calculate total nutrition based on the detected quantity
for food_name, count in food_counts.items():
    if food_name in nutrition_data:
        food_nutrition = nutrition_data[food_name]
        total_calories = food_nutrition['calories'] * count
        total_fat = food_nutrition['fat'] * count
        total_protein = food_nutrition['protein'] * count
        total_sugar = food_nutrition['sugar'] * count

        # Display results
        print(f"Predicted Food: {food_name.capitalize()}")
        print(f"Number of Food Items: {count}")
        print(f"Total Calories: {total_calories} kcal")
        print(f"Total Fat: {total_fat} g")
        print(f"Total Protein: {total_protein} g")
        print(f"Total Sugar: {total_sugar} g")
    else:
        print(f"Nutrition data not available for {food_name}.")


Using cache found in C:\Users\ADMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-12 Python-3.11.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Nutrition data not available for carrot.


In [1]:
import numpy as np
import requests
from PIL import Image
import torch  # We need torch to load YOLOv5
import json  # To load the nutrition data

# Load the nutrition data (replace this with the actual path to your nutrition data file)
def load_nutrition_info(json_file='nutritional_data.json'):
    with open(json_file, 'r') as file:
        nutrition_data = json.load(file)
    return nutrition_data

# Load an image from URL and preprocess it
def load_image_from_url(img_url):
    # Fetch the image from the URL
    response = requests.get(img_url, stream=True)
    img = Image.open(response.raw)
    
    return img

# Load YOLOv5 model (use a custom-trained YOLOv5 model if you have one)
def load_yolov5_model(model_path='yolov5s.pt'):  # yolov5s is a small, pre-trained model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)  # Load custom model
    return model

# Predict the food type, quantity, and total nutrition from an image using YOLOv5
def predict_food_and_quantity(img_url, model, nutrition_data, class_labels):
    # Load and preprocess the image
    img = load_image_from_url(img_url)
    
    # Perform inference with YOLOv5
    results = model(img)  # Inference
    
    # Get the results from YOLO (bounding boxes, labels, and confidence scores)
    detected_classes = results.names  # Class names predicted by YOLO
    detected_boxes = results.xywh[0]  # Bounding boxes (x, y, width, height)
    confidences = results.xywh[0][:, 4]  # Confidence scores for each detection
    class_indices = results.xywh[0][:, 5].tolist()  # Class indices for each detected object
    
    # Count and calculate nutrition for each detected food item
    food_count = {}
    
    for idx, cls_idx in enumerate(class_indices):
        food_name = detected_classes[cls_idx]
        
        # Increase the count for each detected food type
        if food_name in food_count:
            food_count[food_name] += 1
        else:
            food_count[food_name] = 1
    
    # Calculate nutrition for each food detected
    for food_name, count in food_count.items():
        food_nutrition = nutrition_data.get(food_name, None)
        
        if food_nutrition:
            total_calories = food_nutrition['calories'] * count
            total_fat = food_nutrition['fat'] * count
            total_protein = food_nutrition['protein'] * count
            total_sugar = food_nutrition['sugar'] * count
            
            # Display the results
            print(f"Predicted Food: {food_name.capitalize()}")
            print(f"Count: {count}")
            print(f"Total Calories: {total_calories} kcal")
            print(f"Total Fat: {total_fat} g")
            print(f"Total Protein: {total_protein} g")
            print(f"Total Sugar: {total_sugar} g")
        else:
            print(f"Nutrition data not available for {food_name}.")


img_url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREEj1k_FQp0Tk0PhjSciVzjpRs4QibES_TRw&s'

# Example class labels (ensure you have the correct mapping for your food classes)
class_labels = {'ahmok': 0, 'apple': 1, 'orange': 2}  # Update this based on your actual classes


nutrition_data = load_nutrition_info('nutritional_data.json')

# Load the YOLOv5 model
model = load_yolov5_model('yolov5s.pt')  # Replace with your custom-trained model if available

predict_food_and_quantity(img_url, model, nutrition_data, class_labels)


Using cache found in C:\Users\ADMIN/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-2-12 Python-3.11.5 torch-2.2.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Predicted Food: Apple
Count: 1
Total Calories: 95 kcal
Total Fat: 0.3 g
Total Protein: 0.5 g
Total Sugar: 19 g


In [10]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import os
import numpy as np

# Define the augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),  # Flip horizontally with 50% chance
    A.RandomBrightnessContrast(p=0.5),  # Adjust brightness and contrast
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7),  # Random shift, scale, rotate
    A.GaussianBlur(p=0.3),  # Apply Gaussian blur randomly
    # A.RandomSizedCrop(min_max_height=(120, 150), height=150, width=150, p=0.5),  # Random crop & resize
    A.RandomShadow(p=0.3),  # Simulate lighting conditions
    A.RandomRain(p=0.2),  # Simulate rainy conditions
])

# Paths
input_folder = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train/chikhen_rice"
output_folder = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/augmented/chikhen_rice"

# Create output folder if not exist
os.makedirs(output_folder, exist_ok=True)

# Number of augmented images per original image
num_augmentations = 5  # Change this to control the number of variations

# Process each image in the input folder
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)

    # Read the image
    image = cv2.imread(img_path)
    if image is None:
        print(f"Skipping {img_name}, unable to read.")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for i in range(num_augmentations):  # Create multiple versions
        # Apply augmentation
        augmented = transform(image=image)["image"]

        # Convert back to OpenCV format (denormalize & scale)
        augmented = np.clip((augmented * 255), 0, 255).astype(np.uint8)

        # Save augmented image
        output_path = os.path.join(output_folder, f"aug_{i}_{img_name}")
        cv2.imwrite(output_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

print(f"Data augmentation completed! {num_augmentations} new images per original image.")


Data augmentation completed! 5 new images per original image.


NameError: name 'dct' is not defined

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Using a pre-trained model (e.g., ResNet)
        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(512, 256)  # Output feature vector

    def forward_once(self, x):
        return self.backbone(x)

    def forward(self, x1, x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)
        return F.pairwise_distance(output1, output2)  # Distance between images


In [12]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, distance, label):
        loss = (1-label) * torch.pow(distance, 2) + label * torch.pow(torch.clamp(self.margin - distance, min=0.0), 2)
        return loss.mean()


In [13]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import random

# Custom Dataset for One-Shot Learning
class FoodDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.classes = os.listdir(image_folder)
        self.image_paths = {c: [os.path.join(image_folder, c, img) for img in os.listdir(os.path.join(image_folder, c))] for c in self.classes}

    def __len__(self):
        return sum(len(imgs) for imgs in self.image_paths.values())

    def __getitem__(self, index):
        class1 = random.choice(self.classes)
        image1_path = random.choice(self.image_paths[class1])

        if random.random() > 0.5:
            class2 = class1  # Same class (positive pair)
        else:
            class2 = random.choice(self.classes)  # Different class (negative pair)

        image2_path = random.choice(self.image_paths[class2])

        image1 = Image.open(image1_path).convert("RGB")
        image2 = Image.open(image2_path).convert("RGB")

        if self.transform:
            image1 = self.transform(image1)
            image2 = self.transform(image2)

        label = 1 if class1 == class2 else 0  # 1 for same class, 0 for different
        return image1, image2, torch.tensor(label, dtype=torch.float32)

# Define DataLoader
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

dataset = FoodDataset("D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for img1, img2, label in dataloader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(img1, img2)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\ADMIN/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:13<00:00, 3.53MB/s]


Epoch 1, Loss: 17.2795
Epoch 2, Loss: 0.4586
Epoch 3, Loss: 1.2489
Epoch 4, Loss: 0.7574
Epoch 5, Loss: 0.4755
Epoch 6, Loss: 0.2640
Epoch 7, Loss: 0.3181
Epoch 8, Loss: 0.4685
Epoch 9, Loss: 0.2666
Epoch 10, Loss: 0.3163


In [ ]:
def predict(model, image_path, reference_folder):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([transforms.Resize((150, 150)), transforms.ToTensor()])
    image = transform(image).unsqueeze(0).to(device)

    min_distance = float('inf')
    best_match = None

    for class_name in os.listdir(reference_folder):
        for ref_image_path in os.listdir(os.path.join(reference_folder, class_name)):
            ref_image = Image.open(os.path.join(reference_folder, class_name, ref_image_path)).convert("RGB")
            ref_image = transform(ref_image).unsqueeze(0).to(device)
            distance = model(image, ref_image).item()

            if distance < min_distance:
                min_distance = distance
                best_match = class_name

    return best_match

# Predict food class
food_class = predict(model, "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/apple\img_21.jpeg", "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train")
print(f"Predicted Food Class: {food_class}")


Predicted Food Class: ahmok


In [13]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Path to the main train folder
train_folder = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train"

# Get all food class names (subdirectories in the train folder)
food_classes = [d for d in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, d))]

# Data Augmentation Generator
datagen = ImageDataGenerator(
    rotation_range=40,       # Random rotation (0-40 degrees)
    width_shift_range=0.2,   # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip images horizontally
    brightness_range=[0.7, 1.3],  # Vary brightness
    fill_mode="nearest"      # Fill empty pixels
)

# Number of new images per class
num_new_images = 100

# Process each food class
for food_class in food_classes:
    input_folder = os.path.join(train_folder, food_class)
    output_folder = os.path.join(train_folder, f"{food_class}_augmented")

    # Create output directory if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # Get list of images in the folder
    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

    count = 0
    for img_file in image_files:
        img_path = os.path.join(input_folder, img_file)
        img = load_img(img_path, target_size=(224, 224))  # Resize to match model input size
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Generate new images
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_folder, save_prefix="aug", save_format="jpg"):
            i += 1
            count += 1
            if count >= num_new_images:
                break  # Stop once we reach 100 images
        if count >= num_new_images:
            break

    print(f"✅ {food_class}: {num_new_images} new images saved in {output_folder}")

print("🎉 Data Augmentation Completed for All Classes!")


✅ Amok: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Amok_augmented
✅ apple: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\apple_augmented
✅ Borbor: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Borbor_augmented
✅ Cha Loc Lac: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Cha Loc Lac_augmented
✅ Chikhen Rice: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Chikhen Rice_augmented
✅ Lemongrass BBQ Beef Skewers: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Lemongrass BBQ Beef Skewers_augmented
✅ Lort Cha: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train\Lort Cha_augmented
✅ Nom Banh Chok: 100 new images saved in D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/tr

In [14]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Path to the main train folder
train_folder = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train"

# Get all food class names (subdirectories in the train folder)
food_classes = [d for d in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, d))]

# Data Augmentation Generator
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest"
)

# Number of new images per class
num_new_images = 100

# Apply augmentation to each food class
for food_class in food_classes:
    class_folder = os.path.join(train_folder, food_class)

    # Get list of images in the folder
    image_files = [f for f in os.listdir(class_folder) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

    count = 0
    for img_file in image_files:
        img_path = os.path.join(class_folder, img_file)
        img = load_img(img_path, target_size=(224, 224))  # Resize to match model input size
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Generate new images and save inside the same folder
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=class_folder, save_prefix="aug", save_format="jpg"):
            i += 1
            count += 1
            if count >= num_new_images:
                break  # Stop once we reach 100 images
        if count >= num_new_images:
            break

    print(f"✅ {food_class}: {num_new_images} new images added!")

print("🎉 Data Augmentation Completed! Now Training the Model...")

# ---- Start Training Model ----
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(food_classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Save the model
model.save("food_classifier.h5")
print("🎉 Model Training Completed & Saved as food_classifier.h5!")


✅ Amok: 100 new images added!
✅ apple: 100 new images added!
✅ Borbor: 100 new images added!
✅ Cha Loc Lac: 100 new images added!
✅ Chikhen Rice: 100 new images added!
✅ Lemongrass BBQ Beef Skewers: 100 new images added!
✅ Lort Cha: 100 new images added!
✅ Nom Banh Chok: 100 new images added!
✅ Omelete Rice with Pork: 100 new images added!
✅ Orange Fruit: 100 new images added!
✅ Red Curry Coconut Wings: 100 new images added!
✅ SamLar KorKour: 100 new images added!
🎉 Data Augmentation Completed! Now Training the Model...
Found 1405 images belonging to 12 classes.
Found 348 images belonging to 12 classes.


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 32s 687ms/step - accuracy: 0.2040 - loss: 3.6503 - val_accuracy: 0.5172 - val_loss: 1.4026
Epoch 2/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 536ms/step - accuracy: 0.6628 - loss: 1.0761 - val_accuracy: 0.8621 - val_loss: 0.5539
Epoch 3/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 572ms/step - accuracy: 0.8906 - loss: 0.4135 - val_accuracy: 0.9454 - val_loss: 0.3438
Epoch 4/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 539ms/step - accuracy: 0.9023 - loss: 0.3705 - val_accuracy: 0.9368 - val_loss: 0.2363
Epoch 5/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 559ms/step - accuracy: 0.9373 - loss: 0.1891 - val_accuracy: 0.9511 - val_loss: 0.2425
Epoch 6/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 543ms/step - accuracy: 0.9399 - loss: 0.1942 - val_accuracy: 0.9282 - val_loss: 0.2974
Epoch 7/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 24s 538ms/step - accuracy: 0.9469 - loss: 0.1855 - val_accuracy: 0.9684 - val_loss: 0.2075
Epoch 8/10
44/44 ━━━━━━━━━━━━━━━━━━━━ 25s 565ms/step - accuracy: 0.9790 - loss: 0.0649 - val_accu

🎉 Model Training Completed & Saved as food_classifier.h5!


In [19]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('food_classifier.h5')

# Load class labels (same order as training)
class_labels = [
    "Amok", "apple", "BorBor", "Cha Loc Lac", "Chikhen Rice", "Lemongrass BBQ Beef Skewers", 
    "Lort Cha", "Nom Banh Chok", "Omelete Rice with Pork", "Orage Fruit", 
    "Red Curry Coconut Wings", "Samlar KorKour"
]

# Load Nutrition Data
def load_nutrition_info(json_file="nutritional_data.json"):
    with open(json_file, "r") as file:
        data = json.load(file)

    # Convert list to dictionary for easy lookup
    return {item["name"]: {"calories": item["calories"], "carbs": item["carbs"], "fat": item["fat"], "protein": item["protein"]} for item in data}

nutrition_data = load_nutrition_info()

# Function to preprocess and predict food from an image
def predict_food(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  # Get index of highest probability class
    food_name = class_labels[predicted_class]

    # Get nutrition info
    nutrition_info = nutrition_data.get(food_name, {"calories": "N/A", "carbs": "N/A", "fat": "N/A", "protein": "N/A"})

    # Print results
    print(f"Predicted Food: {food_name}")
    print(f"Calories: {nutrition_info['calories']} kcal")
    print(f"Carbs: {nutrition_info['carbs']} g")
    print(f"Fat: {nutrition_info['fat']} g")
    print(f"Protein: {nutrition_info['protein']} g")

# Example Test
image_path = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train/Omelete Rice with Pork/Loco_Moco_at_Ethel's_Grill,_Honolulu,_Hawaii.jpg"  # Change this to your test image path

predict_food(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predicted Food: Nom Banh Chok
Calories: 523.31 kcal
Carbs: 29.95 g
Fat: 1.73 g
Protein: 8.11 g


In [25]:
import os
import shutil
import random

# Original dataset folder
dataset_path = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/train"

# New directories for split data
train_dir = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/split/training_set"
val_dir = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/split/validation_set"
test_dir = "D:/I4_Accadamic_Year/Semester1/AI/Group06_FruitClassification/split/testing_set"

# Create directories
for folder in [train_dir, val_dir, test_dir]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Loop through each food class
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    
    if os.path.isdir(category_path):
        images = [f for f in os.listdir(category_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        random.shuffle(images)

        # Define split sizes
        train_split = int(0.7 * len(images))
        val_split = int(0.2 * len(images))

        # Split images
        train_images = images[:train_split]
        val_images = images[train_split:train_split + val_split]
        test_images = images[train_split + val_split:]

        # Create category folders in new directories
        for target_dir in [train_dir, val_dir, test_dir]:
            category_target = os.path.join(target_dir, category)
            if not os.path.exists(category_target):
                os.makedirs(category_target)

        # Move images
        for img in train_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(train_dir, category, img))

        for img in val_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(val_dir, category, img))

        for img in test_images:
            shutil.copy(os.path.join(category_path, img), os.path.join(test_dir, category, img))

print("✅ Dataset successfully split into train (70%), validation (20%), and test (10%) sets!")


✅ Dataset successfully split into train (70%), validation (20%), and test (10%) sets!


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths for split dataset
train_path = "C:/xampp/htdocs/php-practics/fnapp/model/split/training_set"
val_path = "C:/xampp/htdocs/php-practics/fnapp/model/split/testing_set"
test_path = "C:/xampp/htdocs/php-practics/fnapp/model/split/validation_set"

# Data Generators (rescaling)
train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    verbose=1
)

# Save the trained model
model.save("food_classifier_model.h5")

print("✅ Model Training Completed & Saved!")


Found 1221 images belonging to 12 classes.
Found 185 images belonging to 12 classes.
Found 348 images belonging to 12 classes.


c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 31s 731ms/step - accuracy: 0.2304 - loss: 4.1527 - val_accuracy: 0.5081 - val_loss: 1.5272
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 22s 575ms/step - accuracy: 0.5826 - loss: 1.2985 - val_accuracy: 0.7730 - val_loss: 0.7819
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 25s 628ms/step - accuracy: 0.8154 - loss: 0.6077 - val_accuracy: 0.8919 - val_loss: 0.4292
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 24s 605ms/step - accuracy: 0.9291 - loss: 0.3055 - val_accuracy: 0.8757 - val_loss: 0.4839
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 23s 591ms/step - accuracy: 0.9273 - loss: 0.2491 - val_accuracy: 0.8919 - val_loss: 0.4303
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 22s 564ms/step - accuracy: 0.9568 - loss: 0.1638 - val_accuracy: 0.9027 - val_loss: 0.3871
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 21s 545ms/step - accuracy: 0.9623 - loss: 0.1098 - val_accuracy: 0.9081 - val_loss: 0.4549
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 506ms/step - accuracy: 0.9804 - loss: 0.1060 - val_accu

✅ Model Training Completed & Saved!


In [17]:
import tensorflow as tf
import numpy as np
import json
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('food_classifier_model.h5')

# Load class labels (same order as training)
class_labels = [
    "Amok", "apple", "BorBor", "Cha Loc Lac", "Chikhen Rice", "Lemongrass BBQ Beef Skewers", 
    "Lort Cha", "Nom Banh Chok", "Omelete Rice with Pork", "Orage Fruit", 
    "Red Curry Coconut Wings", "Samlar KorKour"
]

# Load Nutrition Data
def load_nutrition_info(json_file="nutritional_data_info.json"):
    with open(json_file, "r") as file:
        data = json.load(file)

    # Convert list to dictionary for easy lookup
    return {item["name"]: {"calories": item["calories"], "carbs": item["carbs"], "fat": item["fat"], "protein": item["protein"]} for item in data}

nutrition_data = load_nutrition_info()

# Function to preprocess and predict food from an image
def predict_food(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Predict
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)  # Get index of highest probability class
    food_name = class_labels[predicted_class]

    # Get nutrition info
    nutrition_info = nutrition_data.get(food_name, {"calories": "N/A", "carbs": "N/A", "fat": "N/A", "protein": "N/A"})

    # Print results
    print(f"Predicted Food: {food_name}")
    print(f"Calories: {nutrition_info['calories']} kcal")
    print(f"Carbs: {nutrition_info['carbs']} g")
    print(f"Fat: {nutrition_info['fat']} g")
    print(f"Protein: {nutrition_info['protein']} g")

# Example Test
image_path = "C:/xampp/htdocs/php-practics/fnapp/model/split/testing_set/Amok/images (60).jpg"  # Change this to your test image path

predict_food(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Food: Amok
Calories: 453 kcal
Carbs: 22 g
Fat: 9 g
Protein: 38 g


In [18]:
import tensorflow as tf

# Load your trained model
model = tf.keras.models.load_model('food_classifier_model.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('food_classifier_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp3g2ednt9\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmp3g2ednt9\assets


Saved artifact at 'C:\Users\ADMIN\AppData\Local\Temp\tmp3g2ednt9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  2046397345488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397342800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397344336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397341840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397336464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397343568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397334544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397346064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397342416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2046397347600: TensorSpec(shape=(), dtype=tf.resource, name=None)
